# 定制模型

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import autokeras as ak

In [3]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[2] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

在本教程中，我们将展示如何使用 AutoModel 自定义您的搜索空间，以及如何将您自己的模块实现为搜索空间。 这个 API 主要面向已经知道他们的模型应该是什么样子的高级用户。

## 自定义搜索空间

首先，让我们看看如何使用 AutoKeras 中的构建块构建以下神经网络。

![](https://tva1.sinaimg.cn/large/008i3skNly1gs9qgphwsrj312q06qmxt.jpg)

我们可以利用 AutoKeras 中的 AutoModel API 来实现如下。 用法与 Keras 函数式 API 相同。 由于这只是一个演示，我们使用了少量的 max_trials 和 epochs。

In [4]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node1 = ak.ConvBlock()(output_node)
output_node2 = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.Merge()([output_node1, output_node2])
output_node = ak.ClassificationHead()(output_node)

auto_model = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

在构建模型时，使用的块需要遵循以下拓扑：预处理器 -> 块 -> 头部。 Normalization 和 ImageAugmentation 是预处理器。 分类头就是头。 其余的是块。

在上面的代码中，我们使用 ak.ResNetBlock(version='v2') 来指定要使用的 ResNet 版本。 还有许多其他参数要为每个构建块指定。 对于大多数参数，如果未指定，它们将自动调整。 有关更多详细信息，请参阅页面底部的文档链接。

然后，我们准备一些数据来运行模型。

In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

(60000, 28, 28)
(60000,)
[5 0 4]


In [10]:
# Feed the AutoModel with training data.
auto_model.fit(x_train[:100], y_train[:100], epochs=1)

In [7]:
# Predict with the best model.
predicted_y = auto_model.predict(x_test)

313/313 [==============================] - 5s 16ms/step


In [11]:
# Evaluate the best model with testing data.
print(auto_model.evaluate(x_test, y_test))

313/313 [==============================] - 9s 24ms/step - loss: 2.2670 - accuracy: 0.1660
[2.2670044898986816, 0.16599999368190765]


对于多输入节点和多头搜索空间，可以参考本节。

## 验证数据

如果您想提供自己的验证数据或更改验证数据的比例，请参阅图像分类、文本分类、结构化数据分类、多任务和多重验证教程的验证数据部分。

## 数据格式

不同类型数据的格式可以参考ImageInput、StructuredDataInput、TextInput、RegressionHead、ClassificationHead的文档。 也可以参考图像分类、文本分类、结构化数据分类教程的数据格式部分。

## 实现新模块

您可以扩展 Block 类以实现您自己的构建块并将其与 AutoModel 一起使用。

第一步是学习如何为 KerasTuner 编写build函数。 您需要覆盖块的build函数。 下面的例子展示了如何实现一个单一的 Dense 层块，它的神经元数量是可调的。

In [18]:
class SingleDenseLayerBlock(ak.Block):
    def build(self, hp, inputs=None):
        # Get the input_node from inputs.
        input_node = tf.nest.flatten(inputs)[0]
        layer = tf.keras.layers.Dense(
            hp.Int("num_units", min_value=32, max_value=512, step=32)
        )
        output_node = layer(input_node)
        return output_node

您可以将其与其他模块连接并将其构建到 AutoModel 中。

In [19]:
# Build the AutoModel
input_node = ak.Input()
output_node = SingleDenseLayerBlock()(input_node)
output_node = ak.RegressionHead()(output_node)

In [20]:
auto_model = ak.AutoModel(input_node, output_node, overwrite=True, max_trials=1)

In [21]:
# Prepare Data
num_instances = 100
x_train = np.random.rand(num_instances, 20).astype(np.float32)
y_train = np.random.rand(num_instances, 1).astype(np.float32)
x_test = np.random.rand(num_instances, 20).astype(np.float32)
y_test = np.random.rand(num_instances, 1).astype(np.float32)

In [22]:
# Train the model
auto_model.fit(x_train, y_train, epochs=1)

Trial 1 Complete [00h 00m 03s]
val_loss: 0.1839132010936737

Best val_loss So Far: 0.1839132010936737
Total elapsed time: 00h 00m 03s
INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - 0s 13ms/step - loss: 3.3517 - mean_squared_error: 3.3517
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [23]:
print(auto_model.evaluate(x_train, y_test))

4/4 [==============================] - 0s 8ms/step - loss: 2.3558 - mean_squared_error: 2.3558
[2.3558390140533447, 2.3558390140533447]


In [26]:
model = auto_model.export_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                672       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
regression_head_1 (Dense)    (None, 1)                 33        
Total params: 705
Trainable params: 705
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.save('model')

INFO:tensorflow:Assets written to: model/assets


In [28]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model

In [29]:
load_model('model').predict(x_test)

array([[-1.0286503 ],
       [-0.9041684 ],
       [-0.43178535],
       [-0.873243  ],
       [-0.48328778],
       [-1.3547837 ],
       [-0.8065924 ],
       [-1.3461953 ],
       [-0.8086336 ],
       [-1.3170116 ],
       [-0.9794483 ],
       [-1.0683401 ],
       [-0.6362567 ],
       [-1.0264895 ],
       [-0.36868915],
       [-0.9245316 ],
       [-1.20373   ],
       [-0.864015  ],
       [-0.68839157],
       [-1.5398457 ],
       [-1.631917  ],
       [-0.48632136],
       [-1.2327219 ],
       [-0.48484433],
       [-0.5243217 ],
       [-0.6760967 ],
       [-1.5428727 ],
       [-0.930148  ],
       [-1.2085894 ],
       [-1.0193168 ],
       [-0.88419235],
       [-1.1695962 ],
       [-0.37846854],
       [-0.8203037 ],
       [-0.76416177],
       [-1.5360143 ],
       [-1.4622806 ],
       [-0.66567326],
       [-1.3033502 ],
       [-0.84644365],
       [-0.59045357],
       [-1.4576485 ],
       [-1.1818261 ],
       [-1.3567686 ],
       [-1.3793154 ],
       [-1